In [2]:
!pip install folium

In [83]:
# imports libraries
import pandas as pd 
import numpy as np 
import datetime as datetime 
import plotly.express as px
import folium
import json
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

import random
import plotly.io as pio
pio.templates.default = "simple_white"

In [42]:
# notebook configurations
pd.set_option('display.max_columns', None)

## Import Clean Data

In [43]:
# read in clean data
fcn_df = pd.read_excel('data/fcn_19_20.xlsx')
cir_df = pd.read_excel('data/cir_19_20.xlsx')
fc_df = pd.read_excel('data/fc_19_20.xlsx')

# merge fc and fcn to create one df
combined_fc_df = fc_df.merge(fcn_df, on = 'fc_num', how = 'left')

# pull in geodata
zip_codes_gpd = gpd.read_file('data/ZIP_CODES.geojson')

In [44]:
def frequency_table(df, groupby, col, name):
    """
    The function asks as an aggregator. By using the split apply combine method, 
    we group the data and produce the counts and percent of totals.
    """
    
    # grouping offense codes into meaningful representations and then
    grouped = df.groupby(groupby)[[col]]\
            .count().reset_index()\
            .rename(columns = {col:f'{name}_count'})\
            .sort_values(by = f'{name}_count', ascending=False)
    
    grouped[f'percent_of_{name}'] = np.round(grouped.iloc[::,1] / grouped.sum()[1],4)
    return grouped.reset_index(drop=True)   

In [45]:
def percent_diff(df, name):
    """
    This fucntion calculates the month percent difference between 2 years.
    This takes the dataframe and the name of the count columns as input to
    create a table that returns the counts, differences, and % difference.
    """
    monthly_count_df = cir_df.groupby(['month','year'])['incident_number']\
            .count()\
            .reset_index()\
            .rename(columns=dict(incident_number=f'{name}_count'))
    
    monthly_count_df['diff'] = monthly_count_df.groupby('month')[f'{name}_count']\
            .diff().fillna("")
    
    monthly_count_df['pct_diff'] = np.round(monthly_count_df.groupby('month')[f'{name}_count']\
                                            .pct_change() *100,2).fillna("")
    
    return monthly_count_df[monthly_count_df.year == 2020]

## Time Series and Correlation

In [46]:
# create time series data frame
fc_ts_df = fc_df.groupby('date').agg({'fc_num':'count'}).reset_index()
cir_ts_df = cir_df.groupby('date').agg({'incident_number':'count'}).reset_index()

# merge both dataframes to create plot (line chart)
merged_ts_df = fc_ts_df.merge(cir_ts_df ,on='date' ,how='outer')

In [47]:
# quick check of the pearson and spearman correlation
display('Pearson Correlation',merged_ts_df.corr(), 'Spearman Correlation',merged_ts_df.corr('spearman'))

'Pearson Correlation'

,fc_num,incident_number
fc_num,1.000000,0.515977
incident_number,0.515977,1.000000


'Spearman Correlation'

,fc_num,incident_number
fc_num,1.000000,0.535924
incident_number,0.535924,1.000000


In [84]:
# time series of incidents and crimes reported
encounter_incidents = px.line(merged_ts_df,
       title='Field Investigation and Crime Report Counts',
       x='date' ,
       y=['fc_num','incident_number'],
       height = 600,
       width = 1000,
       )

encounter_incidents.update_layout(xaxis_title='Time',yaxis_title='Count',legend_title='Variable')
encounter_incidents

#### Discussion

## Frequency Tables

In [49]:
percent_diff(df=cir_df, name='crime')

,month,year,crime_count,diff,pct_diff
1,1,2020,6342,-616.0,-8.85
3,2,2020,5897,-276.0,-4.47
5,3,2020,5513,-1580.0,-22.28
7,4,2020,4387,-2576.0,-37.0
9,5,2020,5439,-2344.0,-30.12
11,6,2020,5925,-1710.0,-22.4
13,7,2020,6236,-1847.0,-22.85
15,8,2020,6533,-1897.0,-22.5
17,9,2020,6484,-1700.0,-20.77
19,10,2020,6766,-355.0,-4.99


In [50]:
# calculate the frequecy table
cir_frequency_table = frequency_table(df=cir_df,groupby='offense_group',col='incident_number',name='incident')
fcn_frequency_table = frequency_table(df=fcn_df, groupby='race_grouped',col='fc_num',name='encounter')
cir_frequency_table_by_year = frequency_table(df = cir_df,groupby='year',col='incident_number',name='incident')

In [51]:
display(cir_frequency_table, fcn_frequency_table, cir_frequency_table_by_year)

,offense_group,incident_count,percent_of_incident
0,PROPERTY THEFT,26538,0.1679
1,MOTORIZED VEHICLE,19496,0.1233
2,INVESTIGATION,18528,0.1172
3,SICK ASSIST,14900,0.0943
4,ASSAULT,11798,0.0746
5,FRAUD,7531,0.0476
6,DISORDERLY CONDUCT,7523,0.0476
7,MISSING PROPERTY,7468,0.0472
8,VANDALISM,7052,0.0446
9,HARASSMENT,6805,0.0430


,race_grouped,encounter_count,percent_of_encounter
0,Black,16344,0.6626
1,White,6697,0.2715
2,Unknown,1290,0.0523
3,Other,337,0.0137


,year,incident_count,percent_of_incident
0,2019,87184,0.5515
1,2020,70894,0.4485


In [52]:
# NOTES: need to add both 2020 and 2019 crime counts
percent_diff(df=cir_df,name='crime')

,month,year,crime_count,diff,pct_diff
1,1,2020,6342,-616.0,-8.85
3,2,2020,5897,-276.0,-4.47
5,3,2020,5513,-1580.0,-22.28
7,4,2020,4387,-2576.0,-37.0
9,5,2020,5439,-2344.0,-30.12
11,6,2020,5925,-1710.0,-22.4
13,7,2020,6236,-1847.0,-22.85
15,8,2020,6533,-1897.0,-22.5
17,9,2020,6484,-1700.0,-20.77
19,10,2020,6766,-355.0,-4.99


In [85]:
top_ten_df = cir_frequency_table[:10].sort_values('incident_count', ascending=True)

top_ten = px.bar(top_ten_df,
            title='Top 10 Offense Groups by Incident Count',
            x='incident_count',
            y= 'offense_group')

top_ten.update_layout(yaxis_title='Offense Group',xaxis_title='Incident Count')
top_ten.show()

In [53]:
fc_df.head()

,Unnamed: 0,fc_num,contact_date,supervisor,street,city,zip,state,circumstance,basis,key_situations,date,month,ZIP5
0,0,FC20000004,2020-01-01 00:00:00,80411.0,NaN,BOSTON,2128,MA,Stopped,Reasonable Suspicion,NaN,2020-01-01,1,02128
1,1,FC20000075,2020-01-01 00:00:00,10163.0,MAGAZINE ST,ROXBURY,2119,MA,Encountered,Encounter,NaN,2020-01-01,1,02119
2,2,FC20000001,2020-01-01 01:30:00,10092.0,DORCHESTER ST,BOSTON,2127,MA,Encountered,Reasonable Suspicion,NaN,2020-01-01,1,02127
3,3,FC20000002,2020-01-01 06:53:00,11586.0,BOYLSTON ST,BOSTON,2199,MA,Encountered,Encounter,Body Worn Camera,2020-01-01,1,02199
4,4,FC20000003,2020-01-01 10:02:00,12283.0,ANNUNCIATION,ROXBURY,2120,MA,Stopped,Probable Cause,"Body Worn Camera, Juvenile, Warrant Arrest",2020-01-01,1,02120


In [69]:
agg_df = fcn_df.groupby(['race_grouped','age_by_decade', 'sex_grouped'])['fc_num'].count().reset_index().rename(columns=dict(fc_num='count'))
agg_df

,race_grouped,age_by_decade,sex_grouped,count
0,Black,"(10.0, 20.0]",Female,314
1,Black,"(10.0, 20.0]",Male,2929
2,Black,"(20.0, 30.0]",Female,658
3,Black,"(20.0, 30.0]",Male,7364
4,Black,"(20.0, 30.0]",Unknown,3
5,Black,"(30.0, 40.0]",Female,273
6,Black,"(30.0, 40.0]",Male,2772
7,Black,"(40.0, 50.0]",Female,142
8,Black,"(40.0, 50.0]",Male,943
9,Black,"(50.0, 60.0]",Female,96


In [88]:
fig = px.bar(agg_df, x='age_by_decade', y='count', facet_col='race_grouped',color='sex_grouped',height=500)

fig.show()

### Discussion
When cutting the data by age and race we see that Black Men between 20-30 are disproportionately policed when compared to their counterparts.

**TODO:** This visual will give us a lot to talk about for age, rage, and gender

## Geospatial

In [16]:
# count total number of field contacts by zipcode
zip_agg = fc_df.groupby('ZIP5')['fc_num'].count().reset_index().rename(columns=dict(fc_num='count'))

# join counts with geodata
zip_agg_extended = zip_codes_gpd.merge(zip_agg, on='ZIP5', how="left")

In [17]:
zip_agg_extended.head()

,OBJECTID,ZIP5,ShapeSTArea,ShapeSTLength,geometry,count
0,1,02134,3.721936e+07,40794.182396,"POLYGON ((-71.12340 42.36421, -71.12345 42.364...",271.0
1,2,02125,6.476052e+07,62224.521440,"POLYGON ((-71.04541 42.32381, -71.04579 42.323...",1038.0
2,3,02110,6.637284e+06,18358.213496,"POLYGON ((-71.05109 42.36418, -71.05109 42.364...",181.0
3,4,02118,3.116158e+07,32353.407618,"POLYGON ((-71.06315 42.34689, -71.06433 42.347...",1921.0
4,5,02126,6.078585e+07,45488.394711,"POLYGON ((-71.09670 42.29095, -71.09692 42.290...",498.0


In [29]:
cir_frequency_table[:5].sort_values('incident_count', ascending=True).columns

Index(['offense_group', 'incident_count', 'percent_of_incident'], dtype='object')

In [63]:
# # create folium map object
# offense_map = folium.Map(location=[42.361145,-71.057083],
#     tiles="CartoDB positron",
#     zoom_start=12,
#     min_zoom=10,
#     max_zoom=20)


# folium.LayerControl().add_to(offense_map)

# # plot individual crime incident reports
# for coord in cir_df[['lat', 'long','offense_group', 'offense_group_hex']].sample(n=25000).values:
#     #generate random hexcode
#     r = lambda: random.randint(0,255)
#     hex = '#%02X%02X%02X' % (r(),r(),r())
    
#     folium.CircleMarker(location=[coord[0],coord[1]],
#     radius=2,
#     tooltip=coord[2],
#     color = coord[3]
#     ).add_to(offense_map)


# offense_map

**📌 Observations:**

Plotting ~25% of the crime incident report gives us insight into where the crimes are happening in Boston. We can see that many of the crimes appear to be densly clustered in South Central Boston. Let's see how this compares with the field contact data.

In [64]:
# # create folium map object
# top_three_list = cir_frequency_table[:3].sort_values('incident_count', ascending=True)['offense_group'].tolist()


# top_three_scatter_map = folium.Map(location=[42.361145,-71.057083],
#     tiles="CartoDB positron",
#     zoom_start=12,
#     min_zoom=10,
#     max_zoom=20)


# folium.LayerControl().add_to(top_three_scatter_map)

# # plot individual crime incident reports
# for coord in cir_df[['lat', 'long','offense_group', 'offense_group_hex']].values:
#     if coord[2] in top_three_list:
#         folium.CircleMarker(location=[coord[0],coord[1]],
#         radius=2,
#         tooltip=coord[2],
#         color = coord[3]
#         ).add_to(top_three_scatter_map)
#     else:
#         pass


# top_three_scatter_map

**📌 Observations:**

When looking at the top three crimes: `MOTORIZED VEHICLE`, `INVESTIGATION` and `PROPERTY_THEFT` we see some very interesting patterns. Much of the property theft is happening in South Boston.

In [66]:
# zipmap = folium.Map(location=[42.361145,-71.057083], zoom_start=12)
# folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(zipmap)

# myscale = (zip_agg_extended['count'].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()

# zipmap.choropleth(
#  geo_data=zip_agg_extended.dropna(),
#  data=zip_agg_extended.dropna(),
#  columns=['ZIP5','count'],
#  key_on="feature.properties.ZIP5",
#  name="field contact counts",
#  fill_color='YlGnBu',
#  threshold_scale=myscale,
#  legend_name='Number of Field Contacts Reported',
#  fill_opacity=0.5,
#  line_opacity=0.2
# )

# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#000000', 
#                                 'color':'#000000', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}

# TT = folium.features.GeoJson(
#     zip_agg_extended.dropna(),
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['ZIP5', 'count'],
#         aliases=['Zip Code: ','Field Contact Count: '],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )
# zipmap.add_child(TT)

# folium.LayerControl().add_to(zipmap)

# zipmap



In [65]:
# # loop through reported crimes and overlay on zipcode map
# for coord in cir_df[['lat', 'long','offense_group', 'offense_group_hex']].values:
#     if coord[2] in top_three_list:
#         folium.CircleMarker(location=[coord[0],coord[1]],
#         radius=2,
#         tooltip=coord[2],
#         color = coord[3]
#         ).add_to(zipmap)
#     else:
#         pass

# zipmap